In [2]:
import pandas as pd
import requests
import time
from datetime import date, datetime

from config import *

Configuration loaded successfully.


In [3]:
date.today().year - 6

2018

# NHTSA Complaints

### Method 1 - By Make, Model, & Model Year

In [2]:
import pandas as pd
import requests
import time
from datetime import date, datetime

from config import *

########################################
# Main function to download complaints
########################################
# Function to get complaints from NHTSA API
def get_complaints(make, model, model_year):
    # Construct the API URL
    url = f"https://api.nhtsa.gov/complaints/complaintsByVehicle?make={make}&model={model}&modelYear={model_year}"
    
    # Make the GET request to the NHTSA API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Return the JSON response
        return response.json()['results']
    elif response.status_code == 400:
        return None
    else:
        # Return an error message
        return None


In [3]:

########################################
# Update Model Years
########################################
# Function to get all Model Years
def get_model_years():
    url = "https://api.nhtsa.gov/products/vehicle/modelYears?issueType=c"
    response = requests.get(url)
    if response.status_code == 200:
        return pd.DataFrame(response.json()['results'])
    else:
        return f"Error: {response.status_code}"


# Fetch model years
model_years = get_model_years()

# Save model years to database
db = pg_connect()
model_years['updated_on'] = date.today()
model_years.to_sql('model_years',db,index=False,if_exists='replace')
db.dispose()


In [18]:

########################################
# Update Makes
########################################
# Function to get all Makes for the Model Year
def get_makes_for_year(year):
    url = f"https://api.nhtsa.gov/products/vehicle/makes?modelYear={year}&issueType=c"
    response = requests.get(url)
    if response.status_code == 200:
        return pd.DataFrame(response.json()['results'])
    else:
        return f"Error: {response.status_code}"

model_years = pg_query(""" 
select
    *
from model_years
where "modelYear"::INT >= 2016
""")
model_years

db = pg_connect()
for year in model_years['modelYear']:
        print(f'Downloading makes for year {year}')
        makes_for_year = get_makes_for_year(year)
        makes_for_year.to_sql('makes_for_year',db,index=False,if_exists='append')

db.dispose()
# Remove duplicate rows from the table
pg_clean_table('makes_for_year')

Duplicates removed from makes_for_year


### Method 3: Model Year -> Makes -> Models

In [7]:
########################################
# Update Models
########################################
# Function to get all Models for the Make and Model Year
def get_models_for_make_year(make, year):
    url = f"https://api.nhtsa.gov/products/vehicle/models?modelYear={year}&make={make}&issueType=c"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['results']
    else:
        return f"Error: {response.status_code}"
    
if 'models_for_make_year' in pg_tables():
    makes_for_year  = pg_query(""" 
    select distinct 
        "modelYear",
        make
    from makes_for_year 
    where "modelYear"::int >= EXTRACT(YEAR from CURRENT_DATE)::INT 
    and make||"modelYear" in (
        select make||"modelYear" from models_for_make_year
        where CURRENT_DATE - updated_on::Date >7
    )
        """)
else:
    makes_for_year  = pg_query(""" 
    select distinct 
        "modelYear",
        make
    from makes_for_year 
    where "modelYear"::int >= 2016
        """)

all_models = []
if len(makes_for_year) > 0:
    makes_for_year = makes_for_year.sample(500)
    for _,row in makes_for_year.iterrows():
        print(f"Downloading {row['make']} {row['modelYear']} models")
        # Download models
        download = get_models_for_make_year(row['make'],str(row['modelYear']))
        # add to list
        print(download)
        all_models.extend(download)
        time.sleep(.5)
if len(all_models) > 0:
    df = pd.DataFrame(all_models)
    # Save to database
    db = pg_connect()
    df['updated_on'] = date.today()
    df.to_sql('models_for_make_year',db,index=False,if_exists='append')
    pg_clean_table('models_for_make_year')

In [23]:
# Function to get all Models for the Make and Model Year
def get_models_for_make_year(make, year):
    url = f"https://api.nhtsa.gov/products/vehicle/models?modelYear={year}&make={make}&issueType=c"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['results']
    else:
        return f"Error: {response.status_code}"

# Download complaints
Now we use the `get_complaints` method we defined in the beginning of this script to download complaints for each make/model/year. We start with the priority download list.

In [43]:
make_model_year = pg_query(""" 
select 
    models_for_make_year.*,
    last_update.updated_on
from models_for_make_year
join (select distinct make, model, "modelYear", updated_on from complaints) last_update
    on last_update.make = models_for_make_year.make
    and last_update.model = models_for_make_year.model
    and last_update."modelYear" = models_for_make_year."modelYear"
where models_for_make_year."modelYear" >= '2016'
and models_for_make_year.make ='TESLA'
""")
make_model_year

,modelYear,make,model,updated_on
0,2024,TESLA,CYBERTRUCK,2024-03-21
1,2019,TESLA,MODEL 3,2024-03-21
2,2020,TESLA,MODEL 3,2024-03-21
3,2021,TESLA,MODEL 3,2024-03-21
4,2022,TESLA,MODEL 3,2024-03-21
5,2023,TESLA,MODEL 3,2024-03-21
6,2021,TESLA,MODEL S,2024-03-21
7,2022,TESLA,MODEL S,2024-03-21
8,2022,TESLA,MODEL X,2024-03-21
9,2020,TESLA,MODEL Y,2024-03-21


In [44]:
def update_complaints(make_model_year):
    all_complaints = []
    for _,row in make_model_year.iterrows():
        print(f'Downloading data for {row['make']} {row['modelYear']} {row['model']}')
        complaints = get_complaints(row['make'],row['model'],row['modelYear'])
        if complaints:
            for c in complaints:
                c['make'] = row['make']
                c['model'] = row['model']
                c['modelYear'] = row['modelYear']
                c['products'] = json.dumps(c['products'])
                c['updated_on'] = date.today()
                all_complaints.append(c)
        time.sleep(.5)

    df = pd.DataFrame(all_complaints)
    df.to_pickle("all_complaints.pkl")

    db = pg_connect()
    # add data to database:
    df.to_sql('complaints',db,index=False,if_exists='append')
    pg_clean_table('complaints')

In [45]:
make_model_year = pg_query(""" 
select 
    models_for_make_year.*,
    last_update.updated_on
from models_for_make_year
join (select distinct make, model, "modelYear", updated_on from complaints) last_update
    on last_update.make = models_for_make_year.make
    and last_update.model = models_for_make_year.model
    and last_update."modelYear" = models_for_make_year."modelYear"
where models_for_make_year."modelYear" >= '2016'
and models_for_make_year.make ='TESLA'
""")

update_complaints(make_model_year)

Duplicates removed from complaints


In [61]:
make_model_year = pg_query(""" 
select 
    models_for_make_year.*,
    last_update.updated_on
from models_for_make_year
join (select distinct make, model, "modelYear", updated_on from complaints) last_update
    on last_update.make = models_for_make_year.make
    and last_update.model = models_for_make_year.model
    and last_update."modelYear" = models_for_make_year."modelYear"
where models_for_make_year."modelYear" >= '2016'
--and CURRENT_DATE - last_update.updated_on::Date > 7
""")

current_update = make_model_year.sample(500)



In [65]:
make_model_year.sample(500)

,modelYear,make,model,updated_on
1260,2021,INDIAN,SCOUT BOBBER,2024-03-21
1185,2022,GENESIS,G70,2024-03-21
1907,2022,TOYOTA,TUNDRA,2024-03-21
579,2019,TRIUMPH,TIGER 1200 XRX,2024-03-21
1759,2021,GULF STREAM,VINTAGE CRUISER,2024-03-21
...,...,...,...,...
1532,2023,ACURA,RDX,2024-03-21
2039,2023,KAWASAKI,ZX-6R,2024-03-21
521,2021,CHEVROLET,CORVETTE,2024-03-21
1871,2021,AUDI,A8,2024-03-21


In [5]:
car_sales = pd.read_csv('car_sales.csv')
import re
car_sales['Automaker'] = car_sales.Automaker.apply(lambda x: re.sub(',$','',x).split(','))

complaints = []
for _,row in car_sales.iterrows():
    complain = 0
    for maker in row['Automaker']:
        c = pg_query(f""" 
select count(*) from complaints where make = '{maker}'
and "modelYear" = '{row['Year']}'
""")
        complain = complain + c['count'][0] 

    complaints.append(complain)

car_sales['complaints'] = complaints
car_sales['percentage'] = car_sales['complaints']/car_sales['Sold Autos']
car_sales = car_sales[car_sales['percentage']>0]

unparsed = []
for _,row in car_sales.iterrows():
    for make in row['Automaker']:
        unparsed.append({
            'parent': row['Umbrella'],
            'make': make,
            'modelYear': row['Year'],
            'parent_autos_sold': row['Sold Autos'],
            'world_rank': row['World Rank'],
            'parent_modelYear_complaints': row['complaints'],
            'parent_modelYear_percentage': row['percentage']
        })

df = pd.DataFrame(unparsed)
# Save to database
db = pg_connect()
df.to_sql('car_sales',db,index=False,if_exists='replace')
db.dispose()

In [6]:
car_sales

,Automaker,Umbrella,Year,Sold Autos,Status,World Rank,complaints,percentage
0,"[TOYOTA, LEXUS, HINO]",Toyota,2023,10307395,↑ 8%,1,431,0.000042
1,"[TOYOTA, LEXUS, HINO]",Toyota,2022,9566961,↑ 1%,1,558,0.000058
2,"[TOYOTA, LEXUS, HINO]",Toyota,2021,9562483,↑ 1%,1,856,0.000090
3,"[VOLKSWAGEN, AUDI, BENTLEY, LAMBORGHINI, DUCAT...",Volkswagen,2023,9239575,↑ 12%,2,251,0.000027
4,"[VOLKSWAGEN, AUDI, BENTLEY, LAMBORGHINI, DUCAT...",Volkswagen,2022,8263104,↓ 7%,2,519,0.000063
5,"[VOLKSWAGEN, AUDI, BENTLEY, LAMBORGHINI, DUCAT...",Volkswagen,2021,8882346,↓ 5%,2,844,0.000095
6,"[HYUNDAI, HYUNDAI TRANSLEAD, KIA, KIA MOTOR]",Hyundai,2023,7302451,↑ 7%,3,999,0.000137
7,"[HYUNDAI, HYUNDAI TRANSLEAD, KIA, KIA MOTOR]",Hyundai,2022,6848198,↑ 3%,3,1560,0.000228
8,"[HYUNDAI, HYUNDAI TRANSLEAD, KIA, KIA MOTOR]",Hyundai,2021,6668037,↑ 5%,3,1102,0.000165
9,"[ALFA ROMEO, CHRYSLER, DODGE, FIAT, JEEP, MASE...",Stellantis,2023,6392600,↑ 6%,4,300,0.000047


In [3]:
car_sales = pd.read_csv('car_sales.csv')
import re
car_sales['Automaker'] = car_sales.Automaker.apply(lambda x: re.sub(',$','',x).split(','))

complaints = []
for _,row in car_sales.iterrows():
    complain = 0
    for maker in row['Automaker']:
        c = pg_query(f""" 
select count(*) from complaints where make = '{maker}'
and "modelYear" = '{row['Year']}'
""")
        complain = complain + c['count'][0] 

    complaints.append(complain)

car_sales['complaints'] = complaints
car_sales['percentage'] = car_sales['complaints']/car_sales['Sold Autos']
car_sales = car_sales[car_sales['percentage']>0]

unparsed = []
for _,row in car_sales.iterrows():
    for make in row['Automaker']:
        unparsed.append({
            'parent': row['Umbrella'],
            'make': make,
            'modelYear': row['Year'],
            'parent_autos_sold': row['Sold Autos'],
            'world_rank': row['World Rank'],
            'parent_modelYear_complaints': row['complaints'],
            'parent_modelYear_percentage': row['percentage']
        })

df = pd.DataFrame(unparsed)
# Save to database
db = pg_connect()
df.to_sql('car_sales',db,index=False,if_exists='replace')
db.dispose()

pg_execute("""
UPDATE car_sales
SET make = REPLACE(make, 'MERCEDES BENZ', 'MERCEDES-BENZ')
WHERE make = 'MERCEDES BENZ';
""")


'done'